In [1]:
#default_exp segment

In [2]:
#export
import sys
sys.path.insert(0,'..')

from mmseg.apis import init_segmentor, inference_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
import numpy as np
from mytools.tools import *
from urllib.request import urlopen
import cv2
from PIL import Image
from food.paths import *

def url_to_image(url):
	# download the image, convert it to a NumPy array, and then read
	# it into OpenCV format
	resp = urlopen(url)
	image = np.asarray(bytearray(resp.read()), dtype="uint8")
	image = cv2.imdecode(image, cv2.IMREAD_COLOR)
	# return the image
	return image

In [4]:
#export
config_file = '../checkpoints/SETR_Naive_ReLeM/SETR_Naive_768x768_80k_base.py'
checkpoint_file = '../checkpoints/SETR_Naive_ReLeM/iter_80000.pth'
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

/home/dima/FoodSeg103-Benchmark-v1/nbs/../mmseg/models/builder.py:59: UserWarning: train_cfg and test_cfg is deprecated, please specify them in model
  warnings.warn(


load pre-trained weight from imagenet21k


In [30]:
#export
def crop_zeros(img,class_mask):
    class_img = np.array(img) * np.stack([class_mask.T]*3).T
    true_points = np.argwhere(class_mask)
    top_left = true_points.min(axis=0)
    bottom_right = true_points.max(axis=0)
    return (class_img[top_left[0]:bottom_right[0]+1,
                      top_left[1]:bottom_right[1]+1]).astype(np.uint8)

In [32]:
#export
def save_segments(url):
    photo_id = url.split('/')[-1]
    img = url_to_image(url)
    mask = inference_segmentor(model, img)[0]
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    classes = np.unique(mask)[1:]
    for c in classes:
        class_mask = np.where(mask==c,1,0)
        Image.fromarray(crop_zeros(img,class_mask)).save(fragment_reference_images_path/f'{photo_id}_{c}.jpg')
    return mask
        

In [34]:
url = 'https://image.shutterstock.com/z/stock-photo-green-salad-with-baked-chicken-breast-at-white-kitchen-table-healthy-food-clean-eating-concept-1932518612.jpg'

In [37]:
save_segments(url)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [36]:
#[p.unlink() for p in fragment_reference_images_path.ls()]

[None, None, None, None, None, None]

In [42]:
!nbdev_build_lib

Converted 00_paths.ipynb.
Converted segment.ipynb.


In [ ]:
foods = pd.read_sql("""select f.* , c.cluster 
                        from food.foods_prompted f
                        join food.foods_clusters c on (c.id = f.id)""",engine)

# foods = foods.drop(columns = ['clip'])
foods = foods.set_index('id')
foods = foods.dropna()